<img style="width: 150px" src="cbpf_logo.png">

<font size="5"> Deep Learning Dissolution Predictor
    
This is Beta V0.1 version of Neural Network classifier for Dissolution prediction.</font>
<font size="2">
    
This solution was developed by the Brazilian Center for Physics Research (CBPF) AI team.
Authors: Bernardo M. Fraga, Luciana O. Dias, Clécio De Bom and CBPF-CENPES collaboration team.
    
**Contact:** debom@cbpf.br</font>

<font size="2">
Cooperation Agreement (TC) # 2017/00485-5
named: Metodologias Petrofísicas do poro ao poço por Imagens e Inteligência Artificial
between  CBPF  and  PETROBRAS/CENPES. 
    
**PI:** Marcio P. de Albuquerque

Please do not remove this disclaimer.</font>

# Notebook para rodar somente as predições de facies de dissolução de uma rede já treinada.
Os pesos da rede podem ser salvos com o callback `ModelCheckpoint` do notebook de treino

In [ ]:
import tensorflow as tf
import keras
import os, copy, sys
sys.path.insert(0, os.path.abspath('BoreholeTools'))
import numpy as np
import matplotlib.pyplot as plt
from sklearn import metrics
from scipy import interp
from matplotlib.lines import Line2D
from scipy.interpolate import interp1d
from keras import losses
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, CSVLogger, EarlyStopping
from tensorflow.keras import utils
from SlidingDataGenerator import SlidingDataGenerator
from BoreholeImporter import Data
from BoreholeProcessingTools import CropData
from utils import yesnoquestion, uniqueStr, divideblocksize, takeClosest, findChangeIntervals, cm_analysis, ismember
import seaborn as sbr
import pandas as pd
from data_loader import load_dataClass
import matplotlib
import deepnets

### Configurar a GPU

In [ ]:
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [ ]:
__currentPath__ = os.getcwd()

Obter os dados. `MakeChannels` controla se os 3 tipos de imagens (original, log10 e filtro gaussiano) serão geradas. Para a rede pré-treinada, deixar como `True`
### Lembrar de trocar o diretório!

In [ ]:
MakeChannels = True
data_dirSave = "/home/dados4T/DeepDissolutionClass/dataJC"

data_dirIMGW1 = os.path.join(data_dirSave, "W1_5315-5738m_AMP_RAW.csv")
data_dirClassW1 = os.path.join(data_dirSave, "W1_5315-5738m_Facies_Olinto.csv")

data_norm = load_dataClass(data_dirIMG=data_dirIMGW1, data_dirClass=data_dirClassW1, data_dirSave=data_dirSave, MakeChannels=MakeChannels)

Separar os 3 filtros, obter as labels

In [ ]:
orig = data_norm["Acoustic"][:,:,0]
log10 = data_norm["Acoustic"][:,:,1]
gauss = data_norm['Acoustic'][:,:,2]

X = [log10, gauss]
YLab = data_norm["FDISSOL"]
YLevels = list(np.unique(YLab))
YNum = list(map(int, YLab))
Y = tf.keras.utils.to_categorical(YNum, num_classes=3)
n_classes = len(YLevels)

### Configuração dos parâmetros. 
Window size deve ser a mesma do treino. A rede já enviada foi treinada com `window_size=41`

In [ ]:
window_size=41
img_channels = 1
__M__ = np.shape(X[0])[0]
bsize = 64
input_sizes = [(img_channels, window_size, np.shape(x)[1]) for x in X]
weight_for_0 = (1 / np.sum(Y[:,0]))*(Y.shape[0])/3.0
weight_for_1 = (1 / np.sum(Y[:,1]))*(Y.shape[0])/3.0
weight_for_2 = (1 / np.sum(Y[:,2]))*(Y.shape[0])/3.0
class_weight = {0: weight_for_0, 1: weight_for_1, 2: weight_for_2}

### Criar o modelo e usar os pesos treinados.
O arquivo *Model_ws41_bs64_loggauss_resnet50* contém os pesos da rede já treinada. Depois de criado o modelo, inserimos os pesos na rede. Para carregar outra rede já treinada, mudar o arquivo dos pesos.

In [ ]:
model = deepnets.ResnetBuilder.build_resnet_50(input_sizes, n_classes)
model.load_weights('./results/models/Model_ws41_bs64_loggauss_resnet50')

Pegar os índices

In [ ]:
trainIdxs,valIdxs,testIdxs = divideblocksize(__M__,100,0.99,0.01,0.0,
                                                     overlapsize=window_size,display=False,silent=True)

### Fazer as predições

In [ ]:
preds = model.predict_sliding(x=X, batch_size=bsize, sliding_window=window_size, indexes=trainIdxs)

## Plots

Curva ROC

In [ ]:
roc_data = dict()
auc = dict()
for i in range(n_classes):
    roc_data[i] = metrics.roc_curve(YTrueTest, preds[:,i], pos_label=i)#, sample_weight=np.array([class_weight[i]] * preds.shape[0]))
    auc[i] = metrics.auc(roc_data[i][0], roc_data[i][1])

f, ax = plt.subplots(figsize=[9,6])
ax.plot(roc_data[0][0], roc_data[0][1], 'k-', label='class 0, AUC = {:4.2f}'.format(auc[0]))
ax.plot(roc_data[1][0], roc_data[1][1], 'b-', label='class 1, AUC = {:4.2f}'.format(auc[1]))
ax.plot(roc_data[2][0], roc_data[2][1], 'r-', label='class 2, AUC = {:4.2f}'.format(auc[2]))
ax.legend(loc='lower right')
plt.show()

Matriz de confusão

In [ ]:
YTrueTest = [YNum[ii] for ii in trainIdxs]
cm_test = cm_analysis(y_true=YTrueTest,
                                 y_pred=[np.argmax(p) for i,p in enumerate(preds)],
                                 filename=os.path.join(__currentPath__,'results','ConfMatrix_Test_w1_2'),
                                 classes=YLevels,
                                 labels=range(len(YLevels)),
                                 figsize=(12,12),
                                 cmap='Purples')